In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    if len(filenames)!=0:
        for i in range(2):
            print(filenames[i])
    else:
        pass"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install mediapipe

In [ ]:
import numpy as np
import tensorflow as tf
import keras 
from keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Input
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
import os
import re
import mediapipe as mp

In [ ]:
tf.__version__

In [ ]:
# initialize mediapipe 
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

bg_image = cv2.imread("../input/blank-img/blank.jpg")

In [ ]:
def remove_background(img):
    frame = cv2.flip(img, 1)
    height , width, channel = frame.shape

    RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # get the result 
    results = selfie_segmentation.process(RGB)

    # extract segmented mask
    mask = results.segmentation_mask

    # it returns true or false where the condition applies in the mask
    condition = np.stack(
      (results.segmentation_mask,) * 3, axis=-1) > 0.6
    
    bg_image = cv2.imread("../input/blank-img/blank.jpg")
    # resize the background image to the same size of the original frame
    bg_image = cv2.resize(bg_image, (width, height))

    # combine frame and background image using the condition
    output_image = np.where(condition, frame, bg_image)

    return output_image
    

In [ ]:
photo="../input/blank-img/seed0014.png"
img=cv2.imread(photo)
img = remove_background(img)
plt.imshow(img)

In [ ]:
def img2sketch(photo, k_size):
    #Read Image
    img=cv2.imread(photo)
    
    img = remove_background(img)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert to Grey Image
    grey_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Invert Image
    invert_img=cv2.bitwise_not(grey_img)
    #invert_img=255-grey_img

    # Blur image
    blur_img=cv2.GaussianBlur(invert_img, (k_size,k_size),0)

    # Invert Blurred Image
    invblur_img=cv2.bitwise_not(blur_img)
    #invblur_img=255-blur_img

    # Sketch Image
    sketch_img=cv2.divide(grey_img,invblur_img, scale=256.0)

    # Save Sketch 
    #cv2.imwrite('sketch.png', sketch_img)
    return sketch_img
    # Display sketch
    #plt.imshow(sketch_img)
    
#Function call
#img2sketch(photo='../input/celeba-dataset/img_align_celeba/img_align_celeba/000022.jpg', k_size=7)


In [ ]:
img_array = []
sketch_array = []
limit=2472
SIZE=256
# 2472
#count=0

for dirname, _, filenames in os.walk('../input/face-mask-lite-dataset/without_mask'):
    
    if len(filenames)!=0:

        for i in range(1,limit+1):

            file_path=os.path.join(dirname, filenames[i])

            image= cv2.imread(file_path)
            
            image=remove_background(image)

            # as opencv load image in bgr format converting it to rgb
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # resizing images 
            image = cv2.resize(image, (SIZE, SIZE))

            # normalizing image 
            image = image.astype('float32') / 255.0

            #appending normal normal image    
            img_array.append(img_to_array(image))


    
  
        for i in range(1,limit+1):

            file_path=os.path.join(dirname, filenames[i])
            
            image=img2sketch(photo=file_path, k_size=7)

            # as opencv load image in bgr format converting it to rgb
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # resizing images 
            image = cv2.resize(image, (SIZE, SIZE))

            # normalizing image 
            image = image.astype('float32') / 255.0
            # appending normal sketch image
            sketch_array.append(img_to_array(image))



In [ ]:
len(img_array)

In [ ]:
len(sketch_array)

In [ ]:
#2472
print("Total number of sketch images:",len(sketch_array))
print("Total number of images:",len(img_array))

In [ ]:
# defining function to plot images pair
def plot_images(image, sketches):
    plt.figure(figsize=(7,7))
    plt.subplot(1,2,1)
    plt.title('Image', color = 'green', fontsize = 20)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.title('Sketches ', color = 'black', fontsize = 20)
    plt.imshow(sketches)
    plt.show()

In [ ]:
ls = [i for i in range(0,65,8)]
for i in ls:
    plot_images(img_array[i],sketch_array[i])

In [ ]:
dist=2000
train_sketch_image = sketch_array[:dist]
train_image = img_array[:dist]
test_sketch_image = sketch_array[dist:]
test_image = img_array[dist:]
# reshaping
train_sketch_image = np.reshape(train_sketch_image,(len(train_sketch_image),SIZE,SIZE,3))
train_image = np.reshape(train_image, (len(train_image),SIZE,SIZE,3))
print('Train color image shape:',train_image.shape)
test_sketch_image = np.reshape(test_sketch_image,(len(test_sketch_image),SIZE,SIZE,3))
test_image = np.reshape(test_image, (len(test_image),SIZE,SIZE,3))
print('Test color image shape',test_image.shape)

In [ ]:
del img_array
del sketch_array

In [ ]:
def downsample(filters, size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(keras.layers.Conv2D(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_batch_normalization:
        downsample.add(keras.layers.BatchNormalization())
    downsample.add(keras.layers.ReLU())
    return downsample

In [ ]:
def upsample(filters, size, apply_dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(keras.layers.Conv2DTranspose(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_dropout:
        upsample.add(tf.keras.layers.Dropout(0.1))
    upsample.add(tf.keras.layers.ReLU()) 
    return upsample

In [ ]:
def model():
    encoder_input = keras.Input(shape = (SIZE, SIZE, 3))
    x = downsample(16, 4, False)(encoder_input)
    x = downsample(32,4)(x)
    x = downsample(64,4,False)(x)
    x = downsample(128,4)(x)
    
    x = downsample(256,4)(x)
   
    encoder_output = downsample(512,4)(x)
    
    decoder_input = upsample(512,4,True)(encoder_output)
    x = upsample(256,4,False)(decoder_input)
    x = upsample(128,4, True)(x)
    x = upsample(64,4)(x)
    x = upsample(32,4)(x)
    x = upsample(16,4)(x)
    x = tf.keras.layers.Conv2DTranspose(8,(2,2),strides = (1,1), padding = 'valid')(x)
    decoder_output = tf.keras.layers.Conv2DTranspose(3,(2,2),strides = (1,1), padding = 'valid')(x)
    
  
    return tf.keras.Model(encoder_input, decoder_output)

In [ ]:
# to get summary of model
model = model()
model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])

model.fit(train_sketch_image, train_image, epochs = 250, verbose = 1)

In [ ]:
def show_images(real,sketch, predicted):
    plt.figure(figsize = (12,12))
    plt.subplot(1,3,1)
    plt.title("Image",fontsize = 15, color = 'Lime')
    plt.imshow(real)
    plt.subplot(1,3,2)
    plt.title("sketch",fontsize = 15, color = 'Blue')
    plt.imshow(sketch)
    plt.subplot(1,3,3)
    plt.title("Predicted",fontsize = 15, color = 'gold')
    plt.imshow(predicted)

In [ ]:
model.save("mask_wala_data.h5")

In [ ]:
ls = [i for i in range(0,95,8)]
for i in ls:
    predicted =np.clip(model.predict(test_sketch_image[i].reshape(1,SIZE,SIZE,3)),0.0,1.0).reshape(SIZE,SIZE,3)
    show_images(test_image[i],test_sketch_image[i],predicted)